In [1]:
import json
import os
import sys
import numpy as np
import time
import urllib.request
import traceback
from transformers import LlamaForCausalLM, LlamaTokenizer
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import BSHTMLLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from bs4 import BeautifulSoup
import requests
import lxml
import logging

import src.analyse as al

C:\Users\shave\.conda\envs\llamacpp2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
no_factors= 3
no_links = 3
analysis_id = 'nymex_crude_oil'
subject = 'nymex crude oil'
model_props = {
        "name": "llama-2-13b-chat.Q8_0.gguf",
        "n_gpu_layers":40,
        "n_batch":1024,
        "n_ctx":4096
    }

def LoadModel(model_props):
    model_path = os.path.join(os.getcwd(), 'models', model_props['name'])
    grammar_path = os.path.join(os.getcwd(), 'models', 'json.gbnf')
    llm = LlamaCpp(model_path=model_path, 
        temperature=0.0, 
        top_p=1,
        n_ctx=model_props['n_ctx'], 
        seed = 42,
        verbose=True, 
        n_gpu_layers=model_props['n_gpu_layers'],
        n_batch=model_props['n_batch'],
        grammar_path=grammar_path
    )
    return llm

In [3]:
model = LoadModel(model_props)

root ::= object 
object ::= [{] ws object_11 [}] ws 
value ::= object | array | string | number | value_6 ws 
array ::= [[] ws array_15 []] ws 
string ::= ["] string_18 ["] ws 
number ::= number_19 number_25 number_29 ws 
value_6 ::= [t] [r] [u] [e] | [f] [a] [l] [s] [e] | [n] [u] [l] [l] 
ws ::= ws_31 
object_8 ::= string [:] ws value object_10 
object_9 ::= [,] ws string [:] ws value 
object_10 ::= object_9 object_10 | 
object_11 ::= object_8 | 
array_12 ::= value array_14 
array_13 ::= [,] ws value 
array_14 ::= array_13 array_14 | 
array_15 ::= array_12 | 
string_16 ::= [^"\] | [\] string_17 
string_17 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
string_18 ::= string_16 string_18 | 
number_19 ::= number_20 number_21 
number_20 ::= [-] | 
number_21 ::= [0-9] | [1-9] number_22 
number_22 ::= [0-9] number_22 | 
number_23 ::= [.] number_24 
number_24 ::= [0-9] number_24 | [0-9] 
number_25 ::= number_23 | 
number_26 ::= [eE] number_27 number_28 
number_27 ::= [-

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 
from_string grammar:



In [5]:
#res = al.FindFactors(subject, no_factors, no_links, model, 'prod', analysis_id)

In [6]:
#res

["OPEC's market and demand structure and demand elasticity",
 'the information efficiency of crude oil inventory positions',
 'speculative factors']

In [4]:
def BINGQ_Effect(subject, factor):
    bq = 'How can ' + factor + ' affect the price of ' + subject
    return bq

def LLMQ_Effect(subject, factor):
    q = 'How could ' + factor + ' cause the price of ' + subject + ' to Increase'
    t = """
Answer the question at the end using the following context.
<context>
{context}
</context>
Q: {question}? A:
"""
    return q, t

def AnalyseEffect(subject, factor, model, config_file, analysis_id):
    bq = BINGQ_Effect(subject, factor)
    q, t = LLMQ_Effect(subject, factor)
    #al.SearchBingGetLinksAndSave(bq, config_file, analysis_id, 'factor0', no_links)
    answer = al.LoadContextAndRunLLM(q, t, model, config_file, analysis_id, 'factor0')
    return answer

In [5]:
res2 = AnalyseEffect(subject, 'OPECs market and demand structure and demand elasticity', model, 'prod', analysis_id)

In [6]:
res2

{'query': 'How could OPECs market and demand structure and demand elasticity cause the price of nymex crude oil to Increase',
 'result': '{\n\n}\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}